In [17]:
import torch
x = torch.rand(5,3)
print(x)

tensor([[0.7444, 0.3982, 0.1289],
        [0.9307, 0.9378, 0.0151],
        [0.9980, 0.6054, 0.0155],
        [0.4619, 0.4631, 0.6935],
        [0.2737, 0.5315, 0.9398]])


In [3]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [13]:
# Download training data from open datasets
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 9912422/9912422 [00:14<00:00, 671220.51it/s] 


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 175810.21it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:02<00:00, 633081.63it/s] 


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 10490379.28it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [14]:
training_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [15]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [18]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [21]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [24]:
epochs = 5
from tqdm import tqdm
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.021090  [   64/60000]
loss: 0.031974  [ 6464/60000]
loss: 0.037573  [12864/60000]
loss: 0.021822  [19264/60000]
loss: 0.037862  [25664/60000]
loss: 0.061932  [32064/60000]
loss: 0.026460  [38464/60000]
loss: 0.062369  [44864/60000]
loss: 0.067753  [51264/60000]
loss: 0.045878  [57664/60000]
Test Error: 
 Accuracy: 97.5%, Avg loss: 0.078799 

Epoch 2
-------------------------------
loss: 0.018331  [   64/60000]
loss: 0.026334  [ 6464/60000]
loss: 0.030966  [12864/60000]
loss: 0.019877  [19264/60000]
loss: 0.027017  [25664/60000]
loss: 0.049544  [32064/60000]
loss: 0.020082  [38464/60000]
loss: 0.046226  [44864/60000]
loss: 0.062287  [51264/60000]
loss: 0.032633  [57664/60000]
Test Error: 
 Accuracy: 97.5%, Avg loss: 0.077201 

Epoch 3
-------------------------------
loss: 0.016668  [   64/60000]
loss: 0.021216  [ 6464/60000]
loss: 0.025967  [12864/60000]
loss: 0.019096  [19264/60000]
loss: 0.020254  [25664/60000]
loss: 0.038594  [32064/600

In [30]:
model.to('cpu')
model.eval()
x, y = test_data[0][0], test_data[0][1]
pred = model(x)
print(f'Predicted: {pred[0].argmax(0)}, Actual: {y}')

Predicted: 7, Actual: 7
